In [2]:
from pyopenms import *
import pyopenms as pms
import pandas as pd
import numpy as np
import os
import glob

In [3]:
import csv
# read tsv file and create list of FeatureFinderMetaboIdentCompound
def metaboTableFromFile(path_to_library_file):
    metaboTable = []
    with open(path_to_library_file, 'r') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        next(tsv_reader) # skip header
        for row in tsv_reader:
            metaboTable.append(FeatureFinderMetaboIdentCompound(
                row[0], # name
                row[1], # sum formula
                float(row[2]), # mass
                [int(charge) for charge in row[3].split(',')], # charges
                [float(rt) for rt in row[4].split(',')], # RTs
                [float(rt_range) for rt_range in row[5].split(',')], # RT ranges
                [float(iso_distrib) for iso_distrib in row[6].split(',')] # isotope distributions
            ))
    return metaboTable

In [4]:
import glob
from pyopenms import *

input_mzml_files=glob.glob("results/interim/*.mzML")
# load ms data from mzML file into MSExperiment
for mzml_file in input_mzml_files:
    spectra = MSExperiment()
    MzMLFile().load(mzml_file, spectra)

    # create FeatureFinderAlgorithmMetaboIdent and assign ms data
    ff = FeatureFinderAlgorithmMetaboIdent()
    ff.setMSData(spectra)

    # read library generate a metabo table with compounds
    metabo_table = metaboTableFromFile('MetaboliteIdentification.tsv')

    # FeatureMap to store results
    fm = FeatureMap()

    # edit some parameters
    params = ff.getParameters()
    params[b'extract:mz_window'] = 10.0 
    #params[b'extract:rt_window'] = 2.0 
    params[b'detect:peak_width'] = 60.0 
    ff.setParameters(params)

    # run the FeatureFinderMetaboIdent with the metabo_table and store results in fm
    ff.run(metabo_table, fm)

    # save FeatureMap to file
    ff_file = os.path.join("results", "", "FFMI", "", "interim", "", 'FFMI_' + os.path.basename(mzml_file)[19:-5] +".featureXML")
    FeatureXMLFile().store(ff_file, fm) 

In [41]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

common_meta_value_types = {
    b'label': 'U30',
    b'spectrum_index': 'i',
    b'score_fit': 'f',
    b'score_correlation': 'f',
    b'FWHM': 'f',
    b'spectrum_native_id': 'U30',
    b"num_of_masstraces" : "f"
}

class FeatureMapDF(FeatureMap):
    def __init__(self):
        super().__init__()
    
    # meta_values = None (default), 'all' or list of meta value names
    def get_df(self, meta_values = None):
        # get all possible meta value keys in a set
        if meta_values == 'all':
            meta_values = set()
            for f in self:
                mvs = []
                f.getKeys(mvs)
                for m in mvs:
                    meta_values.add(m)
        elif not meta_values: # if None, set to empty list
            meta_values = []
        
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f, meta_values)

        def extract_meta_data(f: Feature, meta_values):
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            bb = f.getConvexHull().getBoundingBox2D()
                
            vals = [f.getMetaValue(m) if f.metaValueExists(m) else np.NA for m in meta_values]   # find some NA or None value for numpy
            
            if len(pep) != 0:
                hits = pep[0].getHits()

                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'), ("PeptideRef", 'U200'),
                    ('quality', 'f'), ('intensity', 'f')]
        
        for meta_value in meta_values:
            if meta_value in common_meta_value_types:
                mddtypes.append((meta_value.decode(), common_meta_value_types[meta_value]))
            else:
                mddtypes.append((meta_value.decode(), 'U50'))
        mdarr = np.fromiter(iter=gen(self, extract_meta_data), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz").drop(columns= "sequence")
        #df= df[df["num_of_masstraces"]>=2]
        return df

In [42]:
input_feature_files = glob.glob('results/FFMI/interim/*.featureXML')

for filename in input_feature_files:
    fmap = FeatureMapDF()
    FeatureXMLFile().load(filename, fmap)
    DF= fmap.get_df()
    feature_csv= os.path.join("results", "", "features", "", 'features_' + os.path.basename(filename)[:-10] +"csv")
    DF.to_csv(feature_csv)
    display(DF)

,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
11459127671857102613,1,70.244102,155.077271,42.477543,93.669006,pyracrimycinB_m154.070007_z1_rt69.540001,-16.881098,4.362530e+05
3363150226772522990,1,355.361786,183.177277,337.679047,372.987671,Geosmin_m182.169998_z1_rt383.359985,-54.329575,4.940536e+06
5048912608372591732,1,102.522278,197.117279,84.527397,123.998314,GermicidinA_m196.110001_z1_rt52.169998,-61.253445,8.543452e+05
15411531668345389314,1,82.115005,402.207275,61.583626,106.193642,bioxalomycina2_m401.200012_z1_rt48.18,-12.191136,2.750892e+05
6770694388801677905,1,70.987061,474.207275,42.477543,94.197739,Unknown_m473.200012_z1_rt44.200001,-79.395035,1.815824e+06
15100022326222889741,1,274.098602,566.277283,261.190643,285.558777,Coelichelin_m565.27002_z1_rt268.049988,-39.060341,2.213446e+06
1285461494282572056,1,317.719696,585.357300,295.650482,339.857574,dehydroxynocardamine_m584.349976_z1_rt351.070007,-70.103745,1.674641e+06
9011896231749582534,1,346.321686,587.337280,326.689087,353.707031,desmethylenylnocardamine_m586.330017_z1_rt339....,-0.198578,2.592657e+05
12157264107681046906,1,351.363129,601.357300,299.000214,368.175446,desferrioxamineE_m600.349976_z1_rt329.5,-4.839808,2.660054e+05


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
10117700810010469827,1,72.604446,139.087280,55.205704,106.907097,pyracrimycinA_m138.080002_z1_rt50.82,-80.996307,2.619421e+05
11387178230956187125,1,97.112816,155.077271,81.082703,109.288483,pyracrimycinB_m154.070007_z1_rt69.540001,-17.620161,2.234700e+05
15288141142181144853,1,319.056824,183.177277,297.979095,340.127014,Geosmin_m182.169998_z1_rt383.359985,-56.156315,1.614904e+05
6825529088171901713,1,51.785267,197.117279,42.491520,72.459312,GermicidinA_m196.110001_z1_rt52.169998,-43.685261,4.101783e+05
7113296351905298096,1,71.711853,402.207275,42.491520,97.355576,bioxalomycina2_m401.200012_z1_rt48.18,-5.929616,2.202441e+05
16806047842200317860,1,57.215672,474.207275,42.491520,76.580284,Unknown_m473.200012_z1_rt44.200001,-50.354980,1.455926e+05
2846611955123922843,1,312.178009,566.277283,270.727325,349.555084,Coelichelin_m565.27002_z1_rt268.049988,-31.956520,1.542035e+06
15053112194686636722,1,393.583618,585.357300,370.568237,430.750214,dehydroxynocardamine_m584.349976_z1_rt351.070007,-21.834169,2.240211e+06
2093352674162466434,1,364.231018,587.337280,321.045380,388.977020,desmethylenylnocardamine_m586.330017_z1_rt339....,-0.279649,1.884833e+06


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
9613685229587292943,1,79.886017,139.087280,59.881725,101.015884,pyracrimycinA_m138.080002_z1_rt50.82,-64.998192,1.384341e+05
13209181295648718664,1,73.775002,155.077271,46.436363,116.841034,pyracrimycinB_m154.070007_z1_rt69.540001,-37.249634,1.343955e+05
2239872228208350408,1,103.368980,197.117279,73.185562,123.295319,GermicidinA_m196.110001_z1_rt52.169998,-0.767028,6.484931e+04
14404510359698007219,1,98.282478,402.207275,81.309341,127.994026,bioxalomycina2_m401.200012_z1_rt48.18,-8.094456,3.997710e+05
6113496519086720906,1,283.977081,566.277283,257.681519,296.805176,Coelichelin_m565.27002_z1_rt268.049988,-7.450653,4.583999e+05
3270301333412202238,1,326.679413,585.357300,298.386078,353.342316,dehydroxynocardamine_m584.349976_z1_rt351.070007,-1.080472,4.320068e+05
16734956595799651367,1,345.928162,587.337280,320.525848,391.128540,desmethylenylnocardamine_m586.330017_z1_rt339....,-24.342161,3.201184e+05
11145873795604457413,1,325.827332,601.357300,304.134521,338.404022,desferrioxamineE_m600.349976_z1_rt329.5,-0.395558,6.637718e+05
9215795462151168254,2,361.730743,712.417297,338.730042,384.910156,epemicinB_m1422.819946_z2_rt405.769989,-72.851440,1.004756e+06


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
17194630121218380359,1,73.386551,139.087280,54.294994,94.724815,pyracrimycinA_m138.080002_z1_rt50.82,-61.694229,1.489369e+07
11183284129613938844,1,164.616028,155.077271,108.439804,189.440918,pyracrimycinB_m154.070007_z1_rt69.540001,-129.625107,1.292996e+06
10164709549591400693,1,100.792488,197.117279,75.016129,119.044884,GermicidinA_m196.110001_z1_rt52.169998,-68.532318,9.049885e+06
5938401199219922809,1,65.811455,402.207275,48.722408,69.549774,bioxalomycina2_m401.200012_z1_rt48.18,-0.732255,2.076304e+07
10026416579231831614,1,60.285683,474.207275,42.476677,88.024216,Unknown_m473.200012_z1_rt44.200001,-25.928787,4.525353e+05
16126906571936877693,1,325.569977,561.357300,302.641876,345.544434,DesferrioxamineB_m560.349976_z1_rt225.770004,-58.739285,1.375448e+07
9745940054150291528,1,285.691315,566.277283,253.260010,294.762390,Coelichelin_m565.27002_z1_rt268.049988,-3.636700,6.873722e+05
14186618305890583063,1,353.501312,585.357300,331.537628,364.421356,dehydroxynocardamine_m584.349976_z1_rt351.070007,-46.280464,1.794589e+07
1792501322850853455,1,343.208466,587.337280,317.738037,363.430939,desmethylenylnocardamine_m586.330017_z1_rt339....,-8.631368,1.190687e+06


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
3444428082740617348,1,85.870964,139.087280,66.316689,105.412254,pyracrimycinA_m138.080002_z1_rt50.82,-60.230068,3301.996094
950009390760440717,1,72.472534,155.077271,48.444859,91.389381,pyracrimycinB_m154.070007_z1_rt69.540001,-18.516109,86788.351562
4774266207309125541,1,96.236954,197.117279,77.191650,129.464294,GermicidinA_m196.110001_z1_rt52.169998,-4.415075,71923.726562
10152185128293613528,1,49.560757,402.207275,42.496136,64.951874,bioxalomycina2_m401.200012_z1_rt48.18,-32.597244,5383.445801
16797017436961594541,1,69.517639,474.207275,53.231213,110.523643,Unknown_m473.200012_z1_rt44.200001,-89.306915,23482.949219
3407629498683591545,1,281.365295,566.277283,265.152283,298.538330,Coelichelin_m565.27002_z1_rt268.049988,-52.889606,6864.420410
5005004260092607923,1,372.371979,585.357300,356.649384,383.792511,dehydroxynocardamine_m584.349976_z1_rt351.070007,-42.157593,3796.017334
6477940247095534562,1,332.048950,587.337280,320.421356,345.957153,desmethylenylnocardamine_m586.330017_z1_rt339....,-40.700706,6857.072754
17198079059817209838,1,318.254303,601.357300,294.154694,340.163696,desferrioxamineE_m600.349976_z1_rt329.5,-74.103813,15283.410156


...

,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
14011811948424435539,1,73.027084,139.087280,57.008041,90.125931,pyracrimycinA_m138.080002_z1_rt50.82,-50.558342,1.348485e+05
8070804649934556324,1,76.360924,155.077271,64.103989,91.001648,pyracrimycinB_m154.070007_z1_rt69.540001,-44.941467,3.106287e+04
14160098512061891114,1,79.808052,197.117279,69.078743,89.313362,GermicidinA_m196.110001_z1_rt52.169998,-30.991104,5.687773e+04
13648199729801765996,1,57.573303,402.207275,42.491123,80.521332,bioxalomycina2_m401.200012_z1_rt48.18,-7.501097,4.151139e+05
15726718888048295204,1,74.884407,474.207275,48.811230,99.034492,Unknown_m473.200012_z1_rt44.200001,-77.656113,8.289358e+04
2547099840851687263,1,255.035431,561.357300,232.346375,277.661072,DesferrioxamineB_m560.349976_z1_rt225.770004,-65.400818,9.378903e+04
15854140661191455914,1,332.294952,566.277283,287.635590,384.811951,Coelichelin_m565.27002_z1_rt268.049988,-4.820932,7.113746e+06
4236541517320953083,1,402.664886,585.357300,376.167084,421.792603,dehydroxynocardamine_m584.349976_z1_rt351.070007,-73.836853,5.693580e+05
10041142438925093539,1,366.323303,587.337280,343.502106,389.403503,desmethylenylnocardamine_m586.330017_z1_rt339....,-14.322666,2.916939e+05


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
16724273447514863017,1,140.277618,139.087280,118.672699,162.294830,pyracrimycinA_m138.080002_z1_rt50.82,-16.233719,1.442552e+05
15562384194388852125,1,64.797241,155.077271,44.722866,84.468872,pyracrimycinB_m154.070007_z1_rt69.540001,-9.731040,1.765813e+05
10360784066269496284,1,103.494385,197.117279,76.200264,120.078743,GermicidinA_m196.110001_z1_rt52.169998,-69.370995,1.275319e+05
15797403964616446152,1,62.464302,402.207275,42.922050,80.525688,bioxalomycina2_m401.200012_z1_rt48.18,-14.571487,2.735992e+05
13071358573601440956,1,73.402908,474.207275,50.281002,101.204422,Unknown_m473.200012_z1_rt44.200001,-81.530220,1.804829e+05
5703465346937927564,1,285.013641,566.277283,254.665131,296.961517,Coelichelin_m565.27002_z1_rt268.049988,-67.145576,1.453098e+06
3535997008604169229,1,337.176544,585.357300,316.599091,356.173706,dehydroxynocardamine_m584.349976_z1_rt351.070007,-53.878185,1.673342e+05
12354958143867410757,1,354.519928,587.337280,335.167419,378.548279,desmethylenylnocardamine_m586.330017_z1_rt339....,-13.236961,4.694054e+05
4306578199916175816,1,365.080078,601.357300,327.379425,398.582123,desferrioxamineE_m600.349976_z1_rt329.5,1.122539,8.814393e+05


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
12417503921932033918,1,76.971695,139.087280,53.642136,104.207527,pyracrimycinA_m138.080002_z1_rt50.82,-81.170197,2.381751e+05
15974178714738380275,1,74.570335,155.077271,47.491425,114.067680,pyracrimycinB_m154.070007_z1_rt69.540001,-13.699897,1.328744e+06
4734297395119257701,1,315.435150,183.177277,294.884491,335.976990,Geosmin_m182.169998_z1_rt383.359985,-59.477997,1.657900e+05
12216779279531442020,1,99.254356,197.117279,70.622070,118.447105,GermicidinA_m196.110001_z1_rt52.169998,-76.898743,1.076536e+05
11942519090494499396,1,96.679527,402.207275,78.221054,120.805893,bioxalomycina2_m401.200012_z1_rt48.18,-71.000282,7.402414e+05
1264700637700980943,1,283.280121,566.277283,249.663254,292.692596,Coelichelin_m565.27002_z1_rt268.049988,-0.652998,2.543070e+05
1816670905140521718,1,345.412598,585.357300,319.932129,372.120117,dehydroxynocardamine_m584.349976_z1_rt351.070007,-80.363403,2.175154e+05
6729354815882575526,1,347.075073,587.337280,317.273865,375.877075,desmethylenylnocardamine_m586.330017_z1_rt339....,-90.260086,3.156556e+05
11174783332204892359,1,353.995667,601.357300,317.685394,367.572937,desferrioxamineE_m600.349976_z1_rt329.5,-11.976419,3.650812e+05


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
308745362452689357,1,88.742119,139.087280,55.785667,104.619209,pyracrimycinA_m138.080002_z1_rt50.82,-78.214020,425.942474
8038829464325864468,1,65.625488,155.077271,45.842751,82.750061,pyracrimycinB_m154.070007_z1_rt69.540001,-58.043285,1525.982178
1019147085104152665,1,363.016327,183.177277,349.310547,382.123505,Geosmin_m182.169998_z1_rt383.359985,-52.400795,0.820837
7916386193623386203,1,99.653488,197.117279,69.773186,143.055084,GermicidinA_m196.110001_z1_rt52.169998,-117.067917,134705.593750
9204603681091103434,1,47.253551,402.207275,42.499962,72.258591,bioxalomycina2_m401.200012_z1_rt48.18,-18.720951,14060.240234
9002953703745392477,1,93.668167,474.207275,76.169449,111.400261,Unknown_m473.200012_z1_rt44.200001,-0.189508,3.887032
16541652474882394481,1,289.830475,566.277283,267.156525,307.580017,Coelichelin_m565.27002_z1_rt268.049988,1.336035,177588.593750
2251212215900043343,1,418.248169,585.357300,384.751526,441.622955,dehydroxynocardamine_m584.349976_z1_rt351.070007,-89.313293,1900.120605
6811990806254481922,1,356.636108,587.337280,324.085999,387.558563,desmethylenylnocardamine_m586.330017_z1_rt339....,-100.345856,15115.690430


,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
9969756798966589731,1,116.302643,139.087280,98.264641,134.503479,pyracrimycinA_m138.080002_z1_rt50.82,-55.079002,5.434910e+05
16631805271944539342,1,74.484245,155.077271,49.088108,107.147797,pyracrimycinB_m154.070007_z1_rt69.540001,-91.604553,5.266913e+05
4045017717420309504,1,61.294563,197.117279,53.139259,66.363960,GermicidinA_m196.110001_z1_rt52.169998,-20.383659,5.277281e+05
5171055758445852439,1,92.646782,402.207275,69.354576,118.751755,bioxalomycina2_m401.200012_z1_rt48.18,-81.086052,4.384487e+05
13451612905357591301,1,81.601929,474.207275,60.725574,102.403610,Unknown_m473.200012_z1_rt44.200001,-66.179207,5.348860e+05
14242650317921044209,1,283.584808,566.277283,265.054443,295.343536,Coelichelin_m565.27002_z1_rt268.049988,-0.297457,2.744437e+05
14235529433467785413,1,315.678131,585.357300,293.808746,337.471588,dehydroxynocardamine_m584.349976_z1_rt351.070007,-61.897797,5.258359e+05
11759851278425456442,1,328.224365,587.337280,320.485382,345.872589,desmethylenylnocardamine_m586.330017_z1_rt339....,-37.744534,4.977380e+05
17128934751856556720,1,363.706146,601.357300,311.171844,388.925354,desferrioxamineE_m600.349976_z1_rt329.5,-0.727993,1.256870e+06
